# 1. Import dependencies

In [1]:
!pip install gym[box2d] pyglet

In [2]:
import  gym
from stable_baselines3 import PPO # on fait le choix de prendre cet algorithme parmi (A2C,ACER,DQN,....)
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy # easier to test our model
import os


# 2. Test Environment

In [3]:
environment_name = 'CarRacing-v0'

In [4]:
env = gym.make(environment_name)

/home/uruk380/anaconda3/envs/Reinforcement_Learning_YT_course/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
print(env.action_space)
print(env.observation_space)
# both are box type

Box(-1.0, 1.0, (3,), float32)
Box(0, 255, (96, 96, 3), uint8)


In [6]:
env.render()

In [7]:
env.close()

In [8]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset() # we have an initial set of observation for the environment
    done = False # is our episode is done ?
    score = 0 # compter

    while not done:
        # allow to view the graphical representation of the environment
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode,score))
env.close()

Track generation: 1198..1502 -> 304-tiles track
Episode: 1 Score: -37.293729372937854
Track generation: 1156..1449 -> 293-tiles track
Episode: 2 Score: -31.506849315068973
Track generation: 1039..1311 -> 272-tiles track
Episode: 3 Score: -26.19926199262023
Track generation: 1249..1574 -> 325-tiles track
Episode: 4 Score: -38.27160493827226
Track generation: 1141..1431 -> 290-tiles track
Episode: 5 Score: -34.25605536332228


In [9]:
env.close()

# 3. Train Model

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [11]:
log_path = os.path.join('Training_Self_Driving_Car', 'Logs')
model = PPO('CnnPolicy',env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [12]:
model.learn(total_timesteps=100000)
# we need much more timesteps to obtain a satisfy result
# 2M is pretty good

Track generation: 1253..1570 -> 317-tiles track
Logging to Training_Self_Driving_Car/Logs/PPO_1
Track generation: 1353..1695 -> 342-tiles track
Track generation: 1109..1390 -> 281-tiles track
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -61.8    |
| time/              |          |
|    fps             | 222      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
Track generation: 1177..1475 -> 298-tiles track
Track generation: 1159..1453 -> 294-tiles track
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -51.4       |
| time/                   |             |
|    fps                  | 150         |
|    iterations           | 2           |
|    time_elapsed         | 27          |
|    total_timesteps      | 4096    

# 4. Save Model

In [13]:
ppo_path = os.path.join('Training_Self_Driving_Car', 'Saved Models', 'PPO Driving Model')

In [14]:
model.save(ppo_path)

In [15]:
del model

In [16]:
model = PPO.load(ppo_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.



# 5. Evaluate and Test

In [9]:
 evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [9]:
episodes = 5  # we test the cartpole environnement five times

for episode in range(1,episodes+1):
    obs = env.reset() # we have an initial set of observation for the environment
    done = False # is our episode is done ?
    score = 0 # compter

    while not done:
        # allow to view the graphical representation of the environment
        env.render()
        # We use our agent here to determine the better action
        action, _ = model.predict(obs)
        score += reward
    print('Episode: {} Score: {}'.format(episode,score))
env.close()